Installer Spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Définir les variables d'environnement

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Importer les data


In [4]:
!wget wget https://datasets.imdbws.com/title.basics.tsv.gz


--2025-12-15 08:42:00--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2025-12-15 08:42:00--  https://datasets.imdbws.com/title.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 3.175.207.35, 3.175.207.127, 3.175.207.91, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|3.175.207.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 214986515 (205M) [binary/octet-stream]
Saving to: ‘title.basics.tsv.gz’

title.basics.tsv.gz 100%[===================>] 205.03M   111MB/s    in 1.8s    

2025-12-15 08:42:03 (111 MB/s) - ‘title.basics.tsv.gz’ saved [214986515/214986515]

FINISHED --2025-12-15 08:42:03--
Total wall clock time: 2.4s
Downloaded: 1 files, 205M in 1.8s (111 MB/s)


In [5]:
import gzip,shutil


In [6]:
with gzip.open('title.basics.tsv.gz', 'rb') as f_in:
    with open('title.basics.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

Parcourir les data

In [43]:
# Load data from csv to a dataframe.
# header=True means the first row is a header
# sep=';' means the column are seperated using ''
df_title = spark.read.csv('./title.basics.tsv', header=True, sep="\t")

df_title = df_title.dropna()

df_cleaned = df_title.filter(df_title["endYear"].isNotNull())
df_cleaned = df_title.filter(df_title["endYear"] != "\\N")


df_cleaned = df_title.filter(
    (df_title["endYear"].isNotNull()) & (df_title["endYear"] != "\\N") &
    (df_title["startYear"].isNotNull()) & (df_title["startYear"] != "\\N")
)

df_cleaned = df_cleaned.filter(
    (df_title["runtimeMinutes"] != "\\N") & df_title["runtimeMinutes"].isNotNull()
)

df_cleaned.sort("endYear",ascending=False).show(2)

df_distinct_genres = df_title.select("genres").distinct()

# Affiche le résultat
df_distinct_genres.show(truncate=False)










+----------+------------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|    tconst|   titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+----------+------------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt37046849|    tvSeries|HexVets and Magic...|HexVets and Magic...|      0|     2026|   2030|            25|Adventure,Animati...|
|tt30761334|tvMiniSeries|             OstyFun|          Les Mythos|      0|     2019|   2029|             3|              Comedy|
+----------+------------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
only showing top 2 rows

+---------------------------+
|genres                     |
+---------------------------+
|Comedy,Sport               |
|Action,Adventure,Fantasy   |
|Documentary,Drama,Fantasy  |
|Adult,Comedy,M

Filtrer

In [14]:
# Register Temporary Table
df_title.createOrReplaceTempView("basics")
# Select all data from temp table
spark.sql("select * from basics where genres like '%Short' and startYear > 2000 limit 5 ").show(truncate=False)

+---------+---------+--------------------------------------------------+--------------------------------------------------+-------+---------+-------+--------------+---------------+
|tconst   |titleType|primaryTitle                                      |originalTitle                                     |isAdult|startYear|endYear|runtimeMinutes|genres         |
+---------+---------+--------------------------------------------------+--------------------------------------------------+-------+---------+-------+--------------+---------------+
|tt0038698|short    |A Little Phantasy on a Nineteenth Century Painting|A Little Phantasy on a Nineteenth Century Painting|0      |2025     |\N     |4             |Animation,Short|
|tt0050396|short    |Final Curtain                                     |Final Curtain                                     |0      |2012     |\N     |22            |Horror,Short   |
|tt0052146|short    |Rondo                                             |Rondo                  

In [ ]:
spark.sql("select * from temp where genres like '%Comedy%' AND startYear = '1999' and titleType = 'movie' LIMIT 5").show(truncate=False)

+---------+---------+-------------------+-------------------+-------+---------+-------+--------------+-------------------+
|tconst   |titleType|primaryTitle       |originalTitle      |isAdult|startYear|endYear|runtimeMinutes|genres             |
+---------+---------+-------------------+-------------------+-------+---------+-------+--------------+-------------------+
|tt0112444|movie    |My Teacher's Wife  |My Teacher's Wife  |0      |1999     |\N     |89            |Comedy,Romance     |
|tt0117148|movie    |Naisen logiikka    |Naisen logiikka    |0      |1999     |\N     |70            |Comedy             |
|tt0118665|movie    |Baby Geniuses      |Baby Geniuses      |0      |1999     |\N     |97            |Comedy,Crime,Family|
|tt0118994|movie    |Do You Wanna Dance?|Do You Wanna Dance?|0      |1999     |\N     |104           |Comedy,Romance     |
|tt0119413|movie    |The Spousals of God|As Bodas de Deus   |0      |1999     |\N     |150           |Comedy             |
+---------+-----

Récupérer rating


In [44]:
!wget wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2025-12-15 10:19:55--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2025-12-15 10:19:55--  https://datasets.imdbws.com/title.ratings.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 3.175.207.127, 3.175.207.96, 3.175.207.35, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|3.175.207.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8122231 (7.7M) [binary/octet-stream]
Saving to: ‘title.ratings.tsv.gz’

title.ratings.tsv.g 100%[===================>]   7.75M  --.-KB/s    in 0.1s    

2025-12-15 10:19:55 (73.7 MB/s) - ‘title.ratings.tsv.gz’ saved [8122231/8122231]

FINISHED --2025-12-15 10:19:55--
Total wall clock time: 0.5s
Downloaded: 1 files, 7.7M in 0.1s (73.7 MB/s)


Extraire les ratings

In [45]:
with gzip.open('title.ratings.tsv.gz', 'rb') as f_in:
    with open('title.ratings.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

Afficher Ratings

In [47]:
# Load data from csv to a dataframe.
# header=True means the first row is a header
# sep=';' means the column are seperated using ''
df_ratings = spark.read.csv('./title.ratings.tsv', header=True, sep="\t")
df_ratings.show(2)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2188|
|tt0000002|          5.5|     307|
+---------+-------------+--------+
only showing top 2 rows



Faire une jointure avec basics

In [48]:
## faire une jointure avec basics

df_jointure = df_cleaned

df_joined = df_cleaned.join(
    df_ratings,
    on='tconst',  # Colonne clé commune pour la jointure
    how='inner'  # Type de jointure (inner, left, right, full, semi, anti)
)

# Afficher les 5 premières lignes du résultat
df_joined.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|tt0051286| tvSeries|             Ivanhoe|             Ivanhoe|      0|     1958|   1959|            30|   Adventure,History|          7.1|     414|
|tt0053512| tvSeries|Phillies Jackpot ...|Phillies Jackpot ...|      0|     1959|   1961|            30| Comedy,Family,Sport|          5.3|      17|
|tt0061259| tvSeries|The Guns of Will ...|The Guns of Will ...|      0|     1967|   1969|            30|             Western|          7.6|     506|
|tt0062578| tvSeries|  Land of the Giants|  Land of the Giants|      0|     1968|   1970|            60|  